In [1]:
# python function test
def add(a,b):
    return a+b

def func_call(func, x, y):
    print(f'res: {func(x, y)}')
    
func_call(add, 4, 5)

res: 9


In [1]:
import casadi

def ode(t, x, u):
    dx0 = casadi.tan(x[3]) * x[1] + 2*u[1] - 1
    dx1 = x[0] - casadi.sin(x[1])
    dx2 = 13 * x[3] + u[0] + 1
    dx3 = 2*x[0]
    
    return casadi.vertcat(dx0, dx1, dx2, dx3)

# h means time step, t_n+1 = t_n + h
# @Think: do we need to care about the casadi's format? 
def rk4(ode, t, x, u, h):
    k1 = ode(t, x, u)
    k2 = ode(t+h/2, x+h*k1/2, u)
    k3 = ode(t+h/2, x+h*k2/2, u)
    k4 = ode(t+h, x+h*k3, u)
    
    return x + h/6*(k1+2*k2+2*k3+k4)

# nonlinear optimization framework
ocp = casadi.Opti()

# define decision variables
nx = 4
nu = 2
X = ocp.variable(nx,1)
U = ocp.variable(nu,1)

# define steady−state problem
ocp.subject_to( ode(0,X,U) == 0)

# solve
ocp.solver('ipopt')
SS_sol = ocp.solve()

x_SS = SS_sol.value(X)
u_SS = SS_sol.value(U)

print("Steady state solution: \n x={} \n u={}".format(x_SS, u_SS))


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        8
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        6
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        4
Total number of inequality c

In [2]:
#  casadi ocp contruction
import casadi as cs
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.animation as ana

# parameters
nx = 4
nu = 1
time_horizon = 15
dt = 1/15

# costs
Q = cs.diag([0.1] * nx)
Q_f = cs.diag([0.5] * nx)
R = cs.diag([0.08] * nu)

# constraints 
x_lb = -np.inf * np.ones(nx)
x_ub = np.inf * np.ones(nx)
u_lb = -np.inf * np.ones(nu)
u_ub = np.inf * np.ones(nu)

# targets
x_ref = cs.DM(np.zeros(nx))
u_ref = cs.DM(np.zeros(nu))

total_optimized_var = nx*(time_horizon+1) + nu*time_horizon

# states
# symbolic representation of the variables
x = cs.SX.sym('x', nx)
print(x)
# controls
u = cs.SX.sym('u', nu)

# ode, some dynamic functions
def make_f(x, u):
    ode0 = (1-x[0]*x[1])*x[0] -x[1] +u
    ode1 = U
    ode = cs.vertcat(ode0,ode1,cs.DM.zeros(1,1), cs.DM.zeros(1,1))
    print(ode)

    # continous function
    casadi_func = cs.Function('f', [x,u], ode, ['x', 'u'], ['x_dot'])
    print(casadi_func)
    return casadi_func

# discretize the dynamic function by rk4
def make_rk4():
    pass

# integrator, works same with rk4 discritization
def make_integrator():
    states = cs.SX.sym('x', nx)
    u = cs.SX.sym('u', nu)
    f = make_f(states, u)
    ode = f(x=states, u=u)['x_dot']
    dae = {'x':states, 'p':u, 'ode':ode}
    I = cs.integrator('I', 'cvodes', dae, 0, dt)
    return I

# cost computation
def conpute_stage_cost(x,u, x_ref, u_ref, Q, R):
    x_diff = x - x_ref
    u_diff = u - u_ref
    return cs.dot(Q*x_diff, x_diff) + cs.dot(R*u_diff, u_diff) / 2

def conpute_terminal_cost(x, Q_f, R):
    x_diff = x - x_ref
    return cs.dot(Q_f*x_diff, x_diff) / 2



[x_0, x_1, x_2, x_3]


In [ ]:
#  casadi NMPC controler
def solver():
    pass



In [24]:
import numpy as np
import casadi as ca
import rospy

rate = 50
T = 1/rate
N = 30                  # horizon length
v_max = 3.0             # linear velocity max
omega_max = np.pi/3.0   # angular velocity max

opti = ca.Opti()
# control variables, liear velocity and angular velocity
opt_controls = opti.variable(N, 3)
vx = opt_controls[:, 0]
vy = opt_controls[:, 1]
omega = opt_controls[:, 2]

opt_states = opti.variable(N+1, 3)
x = opt_states[:, 0]
y = opt_states[:, 1]
theta = opt_states[:, 2]

# parameters, these parameters are the reference trajectories of the pose and inputs
opt_u_ref = opti.parameter(N, 3)
opt_x_ref = opti.parameter(N+1, 3)

# create model
f = lambda x_, u_: ca.vertcat(*[u_[0]*ca.cos(x_[2]) - u_[1]*ca.sin(x_[2]),
                                u_[0]*ca.sin(x_[2]) + u_[1]*ca.cos(x_[2]),
                                u_[2]])
f_np = lambda x_, u_: np.array([u_[0]*ca.cos(x_[2]) - u_[1]*ca.sin(x_[2]),
                                u_[0]*ca.sin(x_[2]) + u_[1]*ca.cos(x_[2]),
                                u_[2]])

# initial condition
opti.subject_to(opt_states[0, :] == opt_x_ref[0, :])
for i in range(N):
    x_next = opt_states[i, :] + f(opt_states[i, :], opt_controls[i, :]).T*T
    opti.subject_to(opt_states[i+1, :] == x_next)

# weight matrix
Q = np.diag([10.0, 10.0, 1.0])
R = np.diag([1.0, 1.0, 0.1])

# cost function
obj = 0
for i in range(N):
    state_error_ = opt_states[i, :] - opt_x_ref[i+1, :]
    # control_error_ = opt_controls[i, :] - opt_u_ref[i, :]
    obj = obj + ca.mtimes([state_error_, Q, state_error_.T]) +\
                ca.mtimes([opt_controls[i, :], R, opt_controls[i, :].T])
opti.minimize(obj)

# boundrary and control conditions
import math
opti.subject_to(opti.bounded(-math.inf, x, math.inf))
opti.subject_to(opti.bounded(-math.inf, y, math.inf))
opti.subject_to(opti.bounded(-math.inf, theta, math.inf))
opti.subject_to(opti.bounded(-v_max, vx, v_max))
opti.subject_to(opti.bounded(-v_max, vy, v_max))
opti.subject_to(opti.bounded(-omega_max, omega, omega_max))

opts_setting = {'ipopt.max_iter':2000,
                'ipopt.print_level':0,
                'print_time':0,
                'ipopt.acceptable_tol':1e-8,
                'ipopt.acceptable_obj_change_tol':1e-6}

opti.solver('ipopt', opts_setting)

t0 = 0
u0 = np.zeros((N, 3))
next_controls = np.zeros((N, 3))
next_states = np.zeros((N+1, 3))  #np.tile(current_state, N+1).reshape(N+1, -1)

while not rospy.is_shutdown():
    ## get the current state
    current_state = np.array([odom.pose.pose.position.x,
                        odom.pose.pose.position.y,
                        quaternion2Yaw(odom.pose.pose.orientation)])

    ## estimate the new desired trajectories and controls
    next_trajectories, next_controls = desired_command_and_trajectory(t0, T, current_state, N)
    next_trajectories = correct_state(next_states, next_trajectories)

    print(current_state)
    print(next_states)

    ## set parameter, here only update initial state of x (x0)
    opti.set_value(opt_x_ref, next_trajectories)
    opti.set_value(opt_u_ref, next_controls)

    ## provide the initial guess of the optimization targets
    opti.set_initial(opt_controls, u0.reshape(N, 3))# (N, 3)
    opti.set_initial(opt_states, next_states) # (N+1, 3)

    ## solve the problem once again
    t_ = time.time()
    sol = opti.solve()

    ## obtain the control input
    u_res = sol.value(opt_controls)
    x_m = sol.value(opt_states)

    t0, current_state, u0, next_states = shift(T, t0, current_state, u_res, x_m, f_np)
    
    # Publish velocity
    vel = Twist()
    vel.linear.x = u_res[0,0]
    vel.linear.y = u_res[0,1]
    vel.angular.z = u_res[0,2]
    pub_vel.publish(vel)

    # Publish predict path
    path = Path()
    path.header = odom.header
    for s in next_states:
        pose = PoseStamped()
        pose.header = path.header
        pose.pose.position.x = s[0]
        pose.pose.position.y = s[1]
        path.poses.append(pose)
    pub_pre_path.publish(path)

    r.sleep()


[0.1, 0.1, 0.1, 0.1]